# (Down)loading package

In [3]:
!pip install numpy
!pip install pep517
!pip install pandas
!pip install Bio

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 7.8 MB/s eta 0:00:00
  Created wheel for pandoc: filename=pandoc-2.3-py3-none-any.whl size=33262 sha256=d4615f43e8c35c71a6f64e8857e521ca7ab1ae3ec4cc3d2c0a46b8219022293e
  Stored in directory: /home/strozyk/.cache/pip/wheels/76/27/c2/c26175310aadcb8741b77657a1bb49c50cc7d4cdbf9eee0005
Successfully built pandoc


In [51]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import SeqIO
from Bio import Entrez

Entrez.email = "elian.strozyk@etu.umontpellier.fr"

In [52]:
# All stylins identified and their NCBI ID
# 01, 02, 03, 04, 04bis, 05
NCBI_Gene_ID = ["NM_001162314.1",
                "NM_001162671.2",
                "NM_001161959.2",
                "NM_001172260.1",
                "NM_001172268.2",
                "NM_001163252.1"
               ]

NCBI_Protein_ID = ["NP_001155786.1",
                   "NP_001156143.1",
                   "NP_001155431.1",
                   "NP_001165731.1",
                   "NP_001165739.1",
                   "NP_001156724.1"
                  ]

In [53]:
################ Retrieve DNA sequences
dna_search_results = Entrez.efetch("Nucleotide",
                                   id = ",".join(NCBI_Gene_ID),
                                   rettype = "fasta"
                                  )
dna_records = SeqIO.parse(dna_search_results, "fasta")
SeqIO.write(dna_records, "./Stylin_DNA_sequence.fasta", "fasta")

6

In [54]:
################ Retrieve protein sequences
protein_search_results = Entrez.efetch("Protein",
                                       id = ",".join(NCBI_Protein_ID),
                                       rettype = "fasta"
                                      )
protein_records = SeqIO.parse(protein_search_results, "fasta")
SeqIO.write(protein_records, "./Stylin_protein_sequence.fasta", "fasta")


6

In [56]:
hit_ids = []
for stylin in NCBI_Protein_ID :
    result_handle = NCBIWWW.qblast(program = "blastp",
                                   database = "nr",
                                   sequence = stylin,
                                   entrez_query = "Aphididae[Organism]",
                                   expect = 0.05,
                                   gapcosts = "11 1",
                                   hitlist_size = 200
                                  )
    blast_records = NCBIXML.parse(result_handle)
    for blast_record in blast_records :
        for alignment in blast_record.alignments :
            if alignment.hit_id not in hit_ids :
                hit_ids.append(alignment.hit_id)
    with open("stylin_protein_blast.xml", "w") as fd :
        fd.write(result_handle.read())
    result_handle.close()
    blast_records.close()

KeyboardInterrupt: 

In [ ]:
#record = SeqIO.read("m_cold.fasta", format="fasta")
#result_handle = NCBIWWW.qblast("blastn", "nt", record.format("fasta"))

In [9]:
hit_ids = []
with open("Stylin_protein_sequence.fasta", "r") as fd :
    result_handle = NCBIWWW.qblast(program = "blastp",
                                   database = "nr",
                                   sequence = fd,
                                   composition_based_statistics = 2,
                                   db_genetic_code = None,
                                   endpoints = None,
                                   entrez_query = "Aphididae[Organism]", 
                                   expect = 0.05, ################
                                   filter = "F",
                                   gapcosts = "11 1", ############
                                   genetic_code = 1,
                                   hitlist_size = 500,
                                   i_thresh = None,
                                   layout = None,
                                   lcase_mask = None,
                                   matrix_name = "BLOSUM62",
                                   other_advanced = None,
                                   perc_ident = None,
                                   phi_pattern = None,
                                   query_file = None,
                                   query_believe_defline = None,
                                   searchsp_eff = None,
                                   service = None,
                                   threshold = 21,
                                   ungapped_alignment = None,
                                   word_size = 5, ###################
                                   short_query = None,
                                   alignments = 500,
                                   alignment_view = None,
                                   descriptions = 500,
                                   expect_low = None,
                                   expect_high = None,
                                   format_object = None,
                                   format_type = "XML",
                                   results_file = None
                                  )
    with open("test.xml", "w") as ft :
        ft.write(result_handle.read())
        blast_records = NCBIXML.parse(result_handle)
        with open("Stylin_protein_blast_IDs.txt", "w") as file :
            for blast_record in blast_records:
                for alignment in blast_record.alignments:
                    hit_ids.append(alignment.hit_id)
                    file.write(alignment.hit_id)

ValueError: Your XML file was empty

In [19]:
help(NCBIWWW.qblast)

Help on function qblast in module Bio.Blast.NCBIWWW:

qblast(program, database, sequence, url_base='https://blast.ncbi.nlm.nih.gov/Blast.cgi', auto_format=None, composition_based_statistics=None, db_genetic_code=None, endpoints=None, entrez_query='(none)', expect=10.0, filter=None, gapcosts=None, genetic_code=None, hitlist_size=50, i_thresh=None, layout=None, lcase_mask=None, matrix_name=None, nucl_penalty=None, nucl_reward=None, other_advanced=None, perc_ident=None, phi_pattern=None, query_file=None, query_believe_defline=None, query_from=None, query_to=None, searchsp_eff=None, service=None, threshold=None, ungapped_alignment=None, word_size=None, short_query=None, alignments=500, alignment_view=None, descriptions=500, entrez_links_new_window=None, expect_low=None, expect_high=None, format_entrez_query=None, format_object=None, format_type='XML', ncbi_gi=None, results_file=None, show_overview=None, megablast=None, template_type=None, template_length=None, username='blast', password=No

In [30]:
#result_handle.close()